In [17]:
# 넷플릭스 영화 추천 시스템 구현
import pandas as pd
import numpy as np
from math import sqrt
import warnings
warnings.filterwarnings('ignore')

In [18]:
# 데이터 읽어오기
# 1205 현재 위치 -> 2개 csv 존재, ex07.py
movies = pd.read_csv("./movies.csv")
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [19]:
ratings = pd.read_csv("./ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
1048570,7120,168,5.0,1175543061
1048571,7120,253,4.0,1175542225
1048572,7120,260,5.0,1175542035
1048573,7120,261,4.0,1175543376


In [20]:
# 아이템 기반 협업 필터링 구현
data = pd.merge(ratings, movies, on="movieId") # merge사용해서 공통된 키로 합침
column = ['userId', 'movieId', 'rating', 'title', 'genres']
data = data[column]
data

,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...
1048570,7066,88572,1.5,Fred: The Movie (2010),Comedy
1048571,7066,112412,4.5,Perfect Sisters (2014),Drama|Mystery
1048572,7077,32013,3.5,Loop the Loop (Up and Down) (Horem pádem) (2004),Comedy
1048573,7086,102596,4.5,180° South (180 Degrees South) (180° South: Co...,Documentary|Drama


In [21]:
moviedata = data.pivot_table(index="movieId", columns='userId')['rating']
moviedata

userId,1,2,3,4,5,6,7,8,9,10,...,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,4.0,NaN,NaN,5.0,NaN,4.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,4.5
2,3.5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,NaN,4.0,NaN,NaN,NaN,3.0,3.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# NaN 값을 -1 로 변경
moviedata.fillna(-1, inplace=True)
moviedata

userId,1,2,3,4,5,6,7,8,9,10,...,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120
movieId,,,,,,,,,,,,,,,,,,,,,
1,-1.0,-1.0,4.0,-1.0,-1.0,5.0,-1.0,4.0,-1.0,4.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,4.0,-1.0,5.0,4.5
2,3.5,-1.0,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0
3,-1.0,4.0,-1.0,-1.0,-1.0,3.0,3.0,5.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,4.0,3.5,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130073,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
130219,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
130462,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [23]:
 # kdd 유사도 함수
def sim_distance(data, n1, n2):
    sum = 0
    # 두 사용자가 모두 본 영화를 기준
    for i in data.loc[n1, data.loc[n1, :] >= 0].index:
        if data.loc[n2, i] >= 0:
            # 누적합
            sum += pow(data.loc[n1, i]-data.loc[n2, i], 2)
    return sqrt(1/(sum+1))  # 유사도 형식으로 출력

In [35]:
 # 나와 유사도 가 높은 유저 매칭
def top_match(data, name, rank=5, simf=sim_distance):
    simList = []
    for i in data.index[-10:]:
        if name != i:
            simList.append((simf(data, name, i), i))
        simList.sort()
        simList.reverse()
    return simList[:rank]

In [36]:
def recommendation(data, person, simf=sim_distance):
    # 추천 시스템 함수
    result_top = top_match(data, person, len(data))
    score_dic = {}
    sim_dic = {}
    my_list = []
    for sim, name in result_top:
        if sim < 0:
            continue
        for movie in data.loc[person, data.loc[person, :] < 0].index:
            sim_sum = 0
            if data.loc[name, movie] >= 0:
                sim_sum += data.loc[name, movie]

                score_dic.setdefault(movie, 0)
                score_dic[movie] += sim_sum

                sim_dic.setdefault(movie, 0)
                sim_dic[movie] += sim

    for key in score_dic:
        my_list.append((score_dic[key]/sim_dic[key], key))
    my_list.sort()
    my_list.reverse()
    return my_list

In [37]:

# 20번 user가 안본 영화중에서 추천 점수가 가장 높은 순으로 예상 평점과 영화 제목을 추천(15개까지)
movie_list = []
for rate, m_id in recommendation(moviedata, 20):
    movie_list.append((rate, movies.loc[movies['movieId'] == m_id, 'title'].values[0]))
    
    if len(movie_list) == 15:
        break

print(movie_list[:15])

[(4.5, "Dracula (Bram Stoker's Dracula) (1992)"), (4.0, 'Entity, The (1981)'), (3.5, 'Born Yesterday (1993)'), (3.5, 'Great Muppet Caper, The (1981)'), (3.5, 'Not One Less (Yi ge dou bu neng shao) (1999)'), (3.5, 'Return with Honor (1998)'), (3.0, 'Left Behind II: Tribulation Force (2002)'), (2.5, 'Bully (2001)'), (2.0, 'Cecil B. DeMented (2000)'), (1.0, "Christmas Vacation (National Lampoon's Christmas Vacation) (1989)"), (0.5, 'Here on Earth (2000)')]


In [40]:
movie_list = []
for rate, m_id in recommendation(moviedata, 2):
    movie_list.append((rate, movies.loc[movies['movieId'] == m_id, 'title'].values[0]))
    
    if len(movie_list) == 15:
        break

print(movie_list[:15])

[(4.0, 'Entity, The (1981)'), (3.5, 'Not One Less (Yi ge dou bu neng shao) (1999)'), (3.0, 'Left Behind II: Tribulation Force (2002)'), (1.8027756377319946, "Christmas Vacation (National Lampoon's Christmas Vacation) (1989)"), (0.5, 'Here on Earth (2000)')]
